# Segmenting and clustering neighbourhoods in Toronto
### (Capstone project course on coursera)

# Part 1

In [4]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bsoup

### Toronto postal codes wikipedia page url:

In [5]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [6]:
page_req = requests.get(url)

### Reading html file and extracting table from it:

In [7]:
page = bsoup(page_req.text, "html.parser")
pgtable = page.table

In [8]:
results = pgtable.find_all("tr")
nrows = len(results)
nrows

181

In [9]:
headers = np.array(results[0].text.split("\n"))[[1,3,5]].tolist()
headers

['Postal Code', 'Borough', 'Neighbourhood']

### Creating dataframe of the table on wiki page:

In [10]:
records = []
n = 1

while n < nrows:
    current_row = results[n].text.split("\n")
    postcode = current_row[1]
    borough = current_row[3]
    nhood = current_row[5]
    records.append((postcode, borough, nhood))
    n = n+1
    
df = pd.DataFrame(records, columns = ["Postalcode", "Borough", "Neighbourhood"])
print(df.head(), "\n", df.shape)

  Postalcode           Borough              Neighbourhood
0        M1A      Not assigned               Not assigned
1        M2A      Not assigned               Not assigned
2        M3A        North York                  Parkwoods
3        M4A        North York           Victoria Village
4        M5A  Downtown Toronto  Regent Park, Harbourfront 
 (180, 3)


In [11]:
df.tail()

,Postalcode,Borough,Neighbourhood
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."
179,M9Z,Not assigned,Not assigned


### Dropping rows for which borough is not assigned:

In [12]:
df1 = df[~df.Borough.str.contains("Not assigned")]
df1 = df1.reset_index(drop = True)
df1.head()

,Postalcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Rows for which neighbourhood is not assigned, it is same as borough:

In [13]:
df1.loc[df1['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df1['Borough']
df1.head()

,Postalcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [14]:
df1['Postalcode'].nunique()

103

There are no repeated postal codes in the dataframe.

In [15]:
df1.head(15)

,Postalcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [16]:
df1.shape

(103, 3)

------End of part 1------

# Part 2: Adding lattitude and longitude to the dataframe

In [24]:
df_pcodes = pd.read_csv("http://cocl.us/Geospatial_data")
df_pcodes.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [29]:
df_pcodes.columns = ['Postalcode', 'Latitude', 'Longitude']
df_pcodes.head()

,Postalcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [31]:
df_lat_lang=pd.merge(df1, df_pcodes, how='right', on = 'Postalcode')
df_lat_lang.head(15)

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [36]:
df_lat_lang.shape

(103, 5)

------ End of part 2 ------